# Create a CIGALE photometry table from the photometry data downloaded from NED for each galaxy

We define the path where the photometry tables from NED was saved and the date when those tables was downloaded

In [6]:
%pylab

Using matplotlib backend: <object object at 0x7fc572291f80>
Populating the interactive namespace from numpy and matplotlib


In [7]:
from datetime import date
import time
import pandas as pd
import numpy as np
import csv
import astropy.units as u

In [8]:
path_phot_data = '../Datos/Phot_Tables/' #In this path wi will save the photometry tables for each object
date_samples = 'Oct_16_2021'

In [9]:
df = pd.read_csv(path_phot_data+'LEDA  165630'+'_'+date_samples+'_phot_tables'+'.csv')

In [10]:
df

,No.,Observed Passband,Photometry Measurement,Uncertainty,Units,Frequency,Flux Density,Upper limit of uncertainty,Lower limit of uncertainty,Upper limit of Flux Density,...,NED Uncertainty,NED Units,Refcode,Significance,Published frequency,Frequency Mode,Coordinates Targeted,Spatial Mode,Qualifiers,Comments
0,1,FUV (GALEX) AB,NaN,>21.2339,mag,1.950000e+15,NaN,NaN,NaN,NaN,...,<1.17E-05,Jy,2012GASC..C...0000S,3 sigma,1538.6 A,Broad-band measurement,212.38039327297 5.1920101373506 (J2000),Flux integrated from map,upper limit inside NUV Kron aperture,From new raw data
1,2,FUV (GALEX) AB,22.0828,+/-0.711414,mag,1.950000e+15,0.000005,3.490000e-06,3.490000e-06,NaN,...,+/-3.49E-06,Jy,2012GASC..C...0000S,uncertainty,1538.6 A,Broad-band measurement,212.38039327297 5.1920101373506 (J2000),Flux in fixed aperture,Flux in 7.5 arcsec diameter aperture,From new raw data
2,3,FUV (GALEX) AB,21.0593,+/-0.0618013,mag,1.950000e+15,0.000014,7.790000e-07,7.790000e-07,NaN,...,+/-7.79E-07,Jy,2012GMSC..C...0000S,uncertainty,1538.6 A,Broad-band measurement,212.38057353833 5.1922816883078 (J2000),Flux integrated from map,Kron flux in elliptical aperture,From new raw data
3,4,FUV (GALEX) AB,22.0570,+/-0.0973424,mag,1.950000e+15,0.000005,4.890000e-07,4.890000e-07,NaN,...,+/-4.89E-07,Jy,2012GMSC..C...0000S,uncertainty,1538.6 A,Broad-band measurement,212.38057353833 5.1922816883078 (J2000),Flux in fixed aperture,Flux in 7.5 arcsec diameter aperture,From new raw data
4,5,NUV (GALEX) AB,20.3448,+/-0.178777,mag,1.290000e+15,0.000026,4.350000e-06,4.350000e-06,NaN,...,+/-4.35E-06,Jy,2012GASC..C...0000S,uncertainty,2315.7 A,Broad-band measurement,212.38039327297 5.1920101373506 (J2000),Flux integrated from map,Kron flux in elliptical aperture,From new raw data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,84,4.8 GHz (VLA),2.7500,NaN,milliJy,4.860000e+09,0.002750,NaN,NaN,NaN,...,NaN,Jy,2006A&A...449..559B,no uncertainty reported,4.86 GHz,Broad-band measurement,14 07 00.77 +05 25 40.7 (J2000),Total flux,NaN,From new raw data
84,85,1.4 GHz (VLA),5.4900,NaN,milliJy,1.420000e+09,0.005490,NaN,NaN,NaN,...,NaN,Jy,2006A&A...449..559B,no uncertainty reported,1.42 GHz,Broad-band measurement,14 07 00.77 +05 25 40.7 (J2000),Total flux,NaN,From new raw data
85,86,1.4GHz,5.2000,+/-0.5,milliJy,1.400000e+09,0.005200,5.000000e-04,5.000000e-04,NaN,...,+/-5.00E-04,Jy,1998AJ....115.1693C,uncertainty,1.40 GHz,Broad-band measurement,14 09 31.41 +05 11 33.9 (J2000),Flux integrated from map,NaN,From new raw data
86,87,1.4 GHz (FIRST),5.5300,NaN,milliJy,1.400000e+09,0.005530,NaN,NaN,NaN,...,NaN,Jy,2017MNRAS.464.1306C,no uncertainty reported,1.4 GHz,Broad-band measurement,212.38024 +05.19203 (J2000),Total flux,Nr. of components = 1,Averaged from previously published data


In [11]:
plt.scatter(df['Frequency'], df['Flux Density'], marker='*', label='NED')
plt.loglog()
plt.xlabel('Wavelength [um]')
plt.ylabel('Flux [Jy]')
plt.legend()